# SOM: 
El algoritmo de Mapas Autoorganizativos (Self-Organizing Maps o SOM) es una técnica de aprendizaje no supervisado utilizada comúnmente para visualizar y organizar datos de alta dimensión en un espacio bidimensional o tridimensional. Aunque el SOM no está diseñado específicamente para clustering, puede utilizarse para agrupar datos de manera efectiva.

In [25]:
from minisom import MiniSom
import numpy as np
from graph_based_corr import *
from utils import * 
import matplotlib.pyplot as plt

# Ejemplo de cómo usar la librería

In [8]:

# Generar datos de ejemplo
np.random.seed(0)
data = np.random.rand(100, 2)

# Definir los parámetros del SOM
som_size = (10, 10)  # Tamaño del mapa SOM
learning_rate = 0.1  # Tasa de aprendizaje
sigma = 1.0  # Ancho del vecindario

# Crear e inicializar el SOM
som = MiniSom(som_size[0], som_size[1], data.shape[1], sigma=sigma, learning_rate=learning_rate)
som.random_weights_init(data)

# Entrenar el SOM
num_epochs = 100
som.train_random(data, num_epochs)

# Asignar cada muestra a un clúster basado en la ubicación en el mapa SOM
cluster_labels = np.array([som.winner(x) for x in data])

# Con 10X_PBMC

In [40]:
x, y = read_data('../../data/10X_PBMC_select_2100.h5')
adata = format_normalize(x, y)
x = adata.X

### Autoencoder: Successfully preprocessed 16653 genes and 2100 cells.


In [41]:
pca = PCA(n_components = 32)
X = pca.fit_transform(x)

In [79]:
# Definir los parámetros del SOM
som_size = (3, 3)  # Tamaño del mapa SOM
learning_rate = 0.1  # Tasa de aprendizaje
sigma = 1.0  # Ancho del vecindario

# Crear e inicializar el SOM
som = MiniSom(som_size[0], som_size[1], X.shape[1], sigma=sigma, learning_rate=learning_rate)
som.random_weights_init(X)

# Entrenar el SOM
num_epochs = 100
som.train_random(X, num_epochs)

# Asignar cada muestra a un clúster basado en la ubicación en el mapa SOM
cluster_labels = np.array([som.winner(x) for x in X])

In [80]:
# Obtener las coordenadas de las neuronas ganadoras para cada muestra
winners = np.array([som.winner(x) for x in X])

# Aplanar las coordenadas para usarlas como características para K-Means
features = winners[:, 0] * som_size[1] + winners[:, 1]

# Realizar clustering con K-Means usando las coordenadas de las neuronas ganadoras
num_clusters = 8
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
cluster_labels = kmeans.fit_predict(features.reshape(-1, 1))

In [81]:
cluster_labels

array([5, 4, 2, ..., 3, 4, 0], dtype=int32)

In [82]:
set(cluster_labels)

{0, 1, 2, 3, 4, 5, 6, 7}

In [83]:
cluster_acc(cluster_labels, y)

0.8119047619047619